In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to your merged model in Google Drive
model_path = "/content/drive/MyDrive/tinyllama-ads-merged"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # or "auto"
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set to eval mode
model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [3]:
# Prompt (same format as training)
prompt = (
    "<s>[INST] <<SYS>>\n"
    "Create a text ad given the following product and description.\n"
    "<</SYS>>\n\n"
    "Product: AromaPlus Coffee Maker\n"
    "Description: A smart drip coffee machine with programmable timer, built-in grinder, and a reusable filter.\n"
    "[/INST] Ad:"
)

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


[INST] <<SYS>>
Create a text ad given the following product and description.
<</SYS>>

Product: AromaPlus Coffee Maker
Description: A smart drip coffee machine with programmable timer, built-in grinder, and a reusable filter.
[/INST] Ad: Program your perfect cup! Our AromaPlus Coffee Machine is compatible with any ground beans, including freshly roasted ones for an extra special taste experience. 😍 Perfect for the tea lover in all of us! Order now! #CoffeeMachine #TeaLovers #AromaplusCoffeemachine 💁‍♀️🥤
